# Usecase: `unfinished new feature` (trunk-based development)

## Scenario

As a programmer, you want to add a new feature. Because your team works trunk-based, you are merging constantly to the master branch. This leads to following problems:

- Your feature is not finished yet, and the usage of it might be problematic
- Your tests of the feature are failing, therefore the tests of other programmers are failing
- You do not know, if the implementation of the feature like this is the best way - neither do the other developer
- The other developer do not want your feature be active directly - due to possible untested side-effects


To avoid this problem, the new feature can be flagged from the beginning.

### Flagging the unittest(s)

In [9]:
import ipytest
ipytest.autoconfig()

from fastfeatureflag.feature_flag import feature_flag

def test_check_fizz():
    fizz__buzz = FizzBuzz()
    assert fizz__buzz.fizz(3) == True
    assert fizz__buzz.fizz(4) == False

# Here starts the first test of the new feature
@feature_flag(name="check_buzz", response=True)
def test_check_buzz():
    fizz_buzz = FizzBuzz()
    assert fizz_buzz.buzz(5) == True
    assert fizz_buzz.buzz(6) == True

Now lets also take a look on the class implementation.

In [7]:
class FizzBuzz:
    def fizz(self, value: int) -> bool:
        if value % 3 == 0:
            return True
        
        return False
    
    @feature_flag(name="check_buzz")
    def buzz(self, value: int) -> bool:
        raise ValueError("Feature not finished, can't work with value.")

In [8]:
ipytest.run()

.                                                                                            [100%]
========================================= warnings summary =========================================
../../fastfeatureflag/feature_flag_configuration.py:57
  /home/grenait/Documents/Git/fastfeatureflag/fastfeatureflag/feature_flag_configuration.py:57: PytestCollectionWarning: cannot collect 'test_check_buzz' because it is not a function.
    def __call__(self, *args: Any, **kwargs: Any) -> Any:

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
1 passed, 1 warning in 0.01s


<ExitCode.OK: 0>

Now, we would like to actually run and test the feature locally. We can easily `activate` the feature flag directly:

In [10]:
import ipytest
ipytest.autoconfig()

from fastfeatureflag.feature_flag import feature_flag

def test_check_fizz():
    fizz__buzz = FizzBuzz()
    assert fizz__buzz.fizz(3) == True
    assert fizz__buzz.fizz(4) == False

# Here starts the first test of the new feature
@feature_flag("on", name="check_buzz", response=True)
def test_check_buzz():
    fizz_buzz = FizzBuzz()
    assert fizz_buzz.buzz(5) == True
    assert fizz_buzz.buzz(6) == True

class FizzBuzz:
    def fizz(self, value: int) -> bool:
        if value % 3 == 0:
            return True
        
        return False
    
    @feature_flag(name="check_buzz")
    def buzz(self, value: int) -> bool:
        raise ValueError("Feature not finished, can't work with value.")

In [11]:
ipytest.run()

.                                                                                            [100%]
========================================= warnings summary =========================================
../../fastfeatureflag/feature_flag_configuration.py:57
  /home/grenait/Documents/Git/fastfeatureflag/fastfeatureflag/feature_flag_configuration.py:57: PytestCollectionWarning: cannot collect 'test_check_buzz' because it is not a function.
    def __call__(self, *args: Any, **kwargs: Any) -> Any:

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
1 passed, 1 warning in 0.03s


<ExitCode.OK: 0>